In [1]:
import numpy as np
from unfolding_helper import *

#helper function to see if to faces are normal to eachother
is_normal = lambda s1, s2 : sum([abs(c1) != abs(c2) for c1, c2 in zip(s1, s2)]) != 0

def strip_n_wings(face, mesh):
    all_normals = list(mesh.face_normals())
    face_normals = all_normals[face.idx()]
    
    #Every face normal to inputted face is strip
    strip = [f.idx() for f in mesh.faces() if is_normal(face_normals, all_normals[f.idx()])]
    
    #Every face that is not strip must be wing
    wings = [f.idx() for f in mesh.faces() if f.idx() not in strip]
    
    return (strip, wings)

In [2]:
def get_best_snw(mesh):
    
    #helper function to select three different faces which are normal to eachother
    def three_normal(mesh):
        all_normals = list(mesh.face_normals())
        faces = []
        
        for f in mesh.faces():
            
            #if current face is not not normal to every face in faces append current face to faces
            if not sum([not is_normal(all_normals[f.idx()], all_normals[ff.idx()]) for ff in faces]):
                faces.append(f)
            
            #if we found three normal faces return them
            if len(faces) == 3:
                return faces
        
        return faces
    
    #List of (strip, wings) tuples from faces out of three_normal()
    snw_list = [strip_n_wings(f, mesh) for f in three_normal(mesh)]
    
    #Rating strips based on size by sorting them
    snw_list.sort(key=lambda snw : len(snw[0]))
    
    return snw_list[-1]
    

In [3]:
def strip_2_tree(mesh, strip, trees = []):
    #Create Rootnode for spanning tree out of first face in strip
    root = node(strip.pop(0))
    curr = root
    
    #small list to store faces which cause errors
    remaining_strip = []
    
    while strip:
        
        #find all adjacent_faces to current face curr                                                                                               
        children = [f for f in strip if f in mesh.get_adjacent_faces_idx(curr.val)]
        
        #break if there are no adjecent faces to the current face left in the strip                                                     
        if not children:                                                                                                                
            break
            
        #choose a child out of possible children, in this case just the first one possible
        child_id = children[0]                                                                                                    
        
        #remove that child from strip and add it to the tree
        strip.remove(child_id)                                                                                                                      
        child_node = node(child_id)                                                                                                                 
        curr.add_child_node(child_node)
        
        #Check if child_node causes overlaps
        if is_unfolding_overlapping(root.unfold(mesh)):
            #if so put child in remaining strip and remove it from tree
            remaining_strip.append(child_node.val)                                                                                      
            root.rem_child_node(child_node)
        else:
            #set curr to the freshly added node
            curr = child_node
    
    #add rest of strip to remaining strip and call strip_2_tree() on it if needed
    if len(strip) > 0 or len(remaining_strip) > 0:                                                                                      
        remaining_strip += strip                                                                                                        
        trees = strip_2_tree(mesh, remaining_strip, trees)                                                                              
    
    #add root (= the current tree) to the trees list
    trees.append(root)
    
    return trees

In [4]:
def attach_wings(mesh, trees, wings): 
    
    #list to store wings which cause errors
    remaining_wings = []                                                                                                                   
    def attach_wing(trees, wing):
        
        #we have to check all trees for all the possible parent faces
        
        #heuristic can be added here to determine best tree
        for tree in trees:
            
            #Check every adjacent face to the wing if it could be a parent
            #heuristic can be added here to determine best parent_face
            for parent_face in mesh.get_adjacent_faces_idx(wing):         
                #if it is: insert it and return
                if tree.has(parent_face):                                                                                                               
                    tree.insert_child(parent_face, node(wing))
                    #check if it causes overlaps
                    if is_unfolding_overlapping(tree.unfold(mesh)):
                        #if so remove it
                        tree.rem_child(wing)
                    else:
                        #otherwise return because wing is stable in tree
                        return
        #if wing can't be attached to any tree append it to remaining_wings
        remaining_wings.append(wing)
        
                        
    #attach all wings
    for wing in wings:                                                                                                                              
        attach_wing(trees, wing)
    
    #treat remaining wings like strips and build seperate trees out of them
    if remaining_wings:
        trees = strip_2_tree(mesh, remaining_wings, trees)
    return trees

In [5]:
mesh = Mesh('Models/L2.obj')
strip, wings = get_best_snw(mesh)
trees = strip_2_tree(mesh, strip)
trees = attach_wings(mesh, trees, wings)

# ATTENTION: global trees list will not be emptied after cell evaluates... can cause confusion --- this is Jupyter Notebook specific
#    -> clear cache
for i, tree in enumerate(trees):
    unfolding = tree.unfold(mesh)
    print(is_unfolding_overlapping(unfolding))
    draw_svg(unfolding, 'Unfolded_SVGs/example' + str(i) + '.svg')

False


  Warning! Material file 'Models/L2.mtl' not found!
Warning! Material 'L2_wire_061135006' not defined in material file.
